In [ ]:
!pip install transformers >> None
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("RussianNLP/ruRoBERTa-large-rucola")

model = AutoModelForSequenceClassification.from_pretrained("RussianNLP/ruRoBERTa-large-rucola")

Downloading:   0%|          | 0.00/792 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
# 1--acceptable
# 0--not acceptable

In [ ]:
from transformers import AutoModelForSequenceClassification, BertTokenizer, AutoTokenizer, DataCollatorWithPadding, \
    TrainingArguments, Trainer
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)
training_args = TrainingArguments(
            output_dir=f"temp",
            evaluation_strategy="epoch",
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            learning_rate=0,
            weight_decay=0,
            num_train_epochs=1,
            warmup_ratio=0.1,
            optim="adamw_torch",
            save_strategy="epoch",
            save_total_limit=1
        )

trainer = Trainer(
            model=model,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=data_collator,
        )

In [ ]:
import pyarrow as pa
import torch 

from datasets import Dataset
def preprocess_function(examples):
    inputs = [example for example in examples["sentence"]]
    model_inputs = tokenizer(inputs, truncation=True, padding=True)
    return model_inputs
def get_prob(arr):
  dataset = []
  for i in range(len(arr)):
    dataset.append({'sentence' : arr[i]})
  dataset = Dataset(pa.Table.from_pylist(dataset))
  dataset = dataset.map(preprocess_function, batched=True)
  outputs = trainer.predict(dataset)
  m = torch.nn.Softmax(dim=1)
  return(m(torch.from_numpy(outputs.predictions))[:, 1])


In [ ]:
beam_1 = get_prob(beams['beam_size = 1'].values)
beam_2 = get_prob(beams['beam_size = 2'].values)
beam_3 = get_prob(beams['beam_size = 3'].values)
beam_4 = get_prob(beams['beam_size = 4'].values)

  0%|          | 0/20 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 20000
  Batch size = 32


  0%|          | 0/20 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 20000
  Batch size = 32


  0%|          | 0/20 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 20000
  Batch size = 32


  0%|          | 0/20 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 20000
  Batch size = 32


In [ ]:
print(beam_1.mean())
print(beam_2.mean())
print(beam_3.mean())
print(beam_4.mean())

tensor(0.9490)
tensor(0.9563)
tensor(0.9579)
tensor(0.9583)


In [ ]:
best_translate = []
for i in tqdm(range(beams.shape[0])):
  probs = [beam_4[i], beam_3[i], beam_2[i], beam_1[i]]
  best_translate.append(beams.iloc[i][f'beam_size = {[4, 3, 2, 1][np.argmax(probs)]}'])

100%|██████████| 20000/20000 [00:05<00:00, 3364.62it/s]


In [ ]:
beams['best_translate'] = best_translate

In [ ]:
beams['beam_1'] = beam_1
beams['beam_2'] = beam_2
beams['beam_3'] = beam_3
beams['beam_4'] = beam_4

In [ ]:
beams.to_csv('beams_probs1.csv', index=False)

In [ ]:
#!pip install evaluate >> None 
#!pip install git+https://github.com/google-research/bleurt.git >> None 

import evaluate
bleu = evaluate.load("bleu")
bleu_beams1 = bleu.compute(predictions=beams_data['best_translate'], references=beams_data['ru'])

In [ ]:
bleu_beams1

{'bleu': 0.38578173983800457,
 'precisions': [0.6842474245063022,
  0.44370952661287527,
  0.32251748544624537,
  0.24231008182310876],
 'brevity_penalty': 0.9829524259723342,
 'length_ratio': 0.9830960983183157,
 'translation_length': 135508,
 'reference_length': 137838}

#### Учтем, что примеры на английском языке могут содержать несколько вариантов перевода на русский:

In [ ]:
from tqdm import tqdm
def get_metric(df):
  arr = []
  for i in tqdm(range(df.shape[0])):
    bl = bleu.compute(predictions=[df['best_translate'].iloc[i]], references=[df['ru'].iloc[i]])
    arr.append(bl['bleu'])
  return arr

In [ ]:
bleu_probs = get_metric(beams_data)

100%|██████████| 20000/20000 [01:36<00:00, 207.12it/s]


In [ ]:
bleu_probs_df = pd.DataFrame()
bleu_probs_df['val'] = bleu_probs
bleu_probs_df['en'] = beams_data['en']
bleu_probs_df = bleu_probs_df.sort_values('val', ascending=False).drop_duplicates('en').sort_index()
bleu_probs_df['val'].mean()

0.3336289798978099

После ранжирования значение метрики оказалось немного хуже, чем значение у лучшей гипотезы. Связано это с тем, что ```ruRoBERTa-large-rucola``` преложения с меньшем количеством слов считает более грамматически правильными.

Иногда примеры с более комплекными и полными переводами при больших значениях ```beam_size``` оказваются длинней и сложней, а следовательно модель ```ruRoBERTa-large-rucola``` может найти в них больше грамматических ошибок.



In [ ]:
! pip install datasets pandas >> None

from datasets import load_dataset
import pandas as pd

raw_data = load_dataset("tatoeba", lang1="en", lang2="tr")

df = pd.DataFrame(raw_data['train']['translation'])
print(f"Dataset contains {len(df)} samples")
df.head(3)

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset contains 672230 samples


,en,tr
0,Let's try something.,Bir şeyler deneyelim!
1,I have to go to sleep.,Yatmaya gitmek zorundayım.
2,I have to go to sleep.,Uyumam lazım.


In [ ]:
 ! pip install scikit-learn >> None
 
from sklearn.model_selection import train_test_split as tts

unique_queries = sorted(df.en.unique().tolist())
 
queries_train_val, queries_test = tts(unique_queries, test_size=0.2, random_state=41)
queries_train, queries_val = tts(queries_train_val, test_size=0.2, random_state=42)

assert set(queries_train) & set(queries_val) == set()
assert set(queries_train) & set(queries_test) == set()
assert set(queries_test) & set(queries_val) == set()

len(queries_train), len(queries_test), len(queries_val)

(409898, 128094, 102475)

In [ ]:
train_df = df.query("en in @queries_train").reset_index(drop=True)
val_df = df.query("en in @queries_val").reset_index(drop=True)
test_df = df.query("en in @queries_test").reset_index(drop=True)

In [ ]:
!pip install transformers[sentencepiece] >> None
import torch
from torch import nn
from torch.nn import functional as F
import transformers as ts
import datasets
from datasets import Dataset

In [ ]:
 from transformers import AutoTokenizer, AutoModel
 tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [ ]:
import pyarrow as pa

In [ ]:
dataset_train = []
for i in range(train_df.shape[0]):
  dataset_train.append({'translation' :  {'en': train_df.iloc[i]['en'], 'tr':  train_df.iloc[i]['tr']}})
dataset_train = Dataset(pa.Table.from_pylist(dataset_train))
dataset_train

Dataset({
    features: ['translation'],
    num_rows: 430040
})

In [ ]:
dataset_val = []
for i in range(val_df.shape[0]):
  dataset_val.append({'translation' :  {'en': val_df.iloc[i]['en'], 'tr':  val_df.iloc[i]['tr']}})
dataset_val = Dataset(pa.Table.from_pylist(dataset_val))
dataset_val

Dataset({
    features: ['translation'],
    num_rows: 107569
})

In [ ]:
dataset_test = []
for i in range(test_df.shape[0]):
  dataset_test.append({'translation' :  {'en': test_df.iloc[i]['en'], 'tr':  test_df.iloc[i]['tr']}})
dataset_test = Dataset(pa.Table.from_pylist(dataset_test))
dataset_test

Dataset({
    features: ['translation'],
    num_rows: 134621
})

In [ ]:
source_lang = "en"
target_lang = "tr"
prefix = "translate English to Turkish: "
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_train = dataset_train.map(preprocess_function, batched=True)
tokenized_test = dataset_test.map(preprocess_function, batched=True)
tokenized_val = dataset_val.map(preprocess_function, batched=True)

  0%|          | 0/431 [00:00<?, ?ba/s]

  0%|          | 0/135 [00:00<?, ?ba/s]

  0%|          | 0/108 [00:00<?, ?ba/s]

In [ ]:
from transformers import  AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 430040
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 26878
  Number of trainable parameters = 300176768
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than us

Epoch,Training Loss,Validation Loss
1,2.488200,1.720350


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Copy vocab file to ./results/checkpoint-500/spiece.model
Deleting older checkpoint [results/checkpoint-4000] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Copy vocab file to ./results/checkpoint-1000/spiece.model
Deleting older checkpoint [results/checkpoint-4500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-15

TrainOutput(global_step=26878, training_loss=3.272248138761545, metrics={'train_runtime': 8661.4751, 'train_samples_per_second': 49.65, 'train_steps_per_second': 3.103, 'total_flos': 1.141587932209152e+16, 'train_loss': 3.272248138761545, 'epoch': 1.0})

In [ ]:
dataset_test_small = []
for i in range(len(test_df.drop_duplicates(subset=['en']).iloc[:20]['en'].values)):
  dataset_test_small.append({'translation' :  {'en': test_df.drop_duplicates(subset=['en']).iloc[:1000]['en'].values[i],
                                               'tr': test_df.drop_duplicates(subset=['en']).iloc[:1000]['tr'].values[i]}})
dataset_test_small = Dataset(pa.Table.from_pylist(dataset_test_small))
dataset_test_small

Dataset({
    features: ['translation'],
    num_rows: 20
})

In [ ]:
tokenized_test_small = dataset_test_small.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_test_small

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 20
})

In [ ]:
outputs = trainer.predict(tokenized_test_small)

The following columns in the test set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 20
  Batch size = 16


In [ ]:
# from transformers import AutoModelForSeq2SeqLM

# model = AutoModelForSeq2SeqLM.from_pretrained("/content/my_model")

In [ ]:
prefix = "translate English to Turkish: "
inp = [prefix + sent for sent in test_df['en'].values]

In [ ]:
prefix = "translate English to Turkish: "
inp = [prefix + sent for sent in test_df['en'].values]
from tqdm import tqdm
tranlated_tr = []
for i in tqdm(range(0,134621, 16)):
  inputs = tokenizer(
        inp[i : i + 16],
        truncation=True,
        return_tensors='pt',
        padding=True).to('cuda')
  translation = model.to('cuda').generate(**inputs)
  tranlated_tr += tokenizer.batch_decode(translation, skip_special_tokens=True)

  0%|          | 0/8414 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
100%|██████████| 8414/8414 [41:31<00:00,  3.38it/s]


In [ ]:
len(tranlated_tr)

134621

In [ ]:
tranlated_tr[-1] = ''

In [ ]:
test_df['tr_transformer'] = tranlated_tr

In [ ]:
test_df.to_csv('tr_1.csv', index=False)

In [ ]:
from google.colab import files
files.download('tr_1.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sentence = "translate English to Turkish: I love you"
input_ids = tokenizer(sentence, return_tensors="pt").input_ids
attention_mask=tokenizer(sentence, return_tensors="pt").attention_mask

In [ ]:
outputs = model.generate(input_ids.to('cuda'))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Seninle seni seviyorum.']

In [ ]:
prefix = "translate English to Turkish: "
inp = [prefix + sent for sent in train_df['en'].values]
from tqdm import tqdm
tranlated_tr = []
for i in tqdm(range(0,409898, 16)):
  inputs = tokenizer(
        inp[i : i + 16],
        truncation=True,
        return_tensors='pt',
        padding=True).to('cuda')
  translation = model.to('cuda').generate(**inputs)
  tranlated_tr += tokenizer.batch_decode(translation, skip_special_tokens=True)

In [ ]:
train_df['tr_transformer'] = tranlated_tr
train_df.to_csv('train_df_tr.csv', index=False)

In [ ]:
from google.colab import files
files.download('train_df_tr.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_df = pd.read_csv('tr_1.csv')
train_df = pd.read_csv('train_df_tr.csv')

In [ ]:
# !pip install evaluate >> None 
# !pip install git+https://github.com/google-research/bleurt.git >> None 

import evaluate
bleu = evaluate.load("bleu")

bleu_results_tr_test = bleu.compute(predictions=test_df['tr_transformer'], references=test_df['tr'])
bleu_results_tr_train = bleu.compute(predictions=train_df['tr_transformer'], references=train_df['tr'])

In [ ]:
print(bleu_results_tr_test)
print(bleu_results_tr_train)

{'bleu': 0.13517668166750468, 'precisions': [0.4658898191229971, 0.18746243629195114, 0.08895945874676973, 0.045675878877176695], 'brevity_penalty': 0.9848784861479639, 'length_ratio': 0.9849916722984825, 'translation_length': 800157, 'reference_length': 812349}
{'bleu': 0.13440915826923946, 'precisions': [0.46583330008316454, 0.1874471509233245, 0.08848504213292711, 0.045021812511103654], 'brevity_penalty': 0.9841885505741071, 'length_ratio': 0.9843122442119563, 'translation_length': 2556378, 'reference_length': 2597121}
